In [14]:

from umap_script import loadHSI, show_clusterable_embedding, visualize_umap, compare_umap    
from sklearn.neighbors import NearestNeighbors
import math

import numpy as np
import random
import matplotlib.pyplot as plt
import umap

In [11]:
salinasA_path = 'data/SalinasA_corrected.mat'
salinasA_gt_path = 'data/SalinasA_gt.mat'

X, M, N, D, HSI, GT, Y, n, K = loadHSI(salinasA_path, salinasA_gt_path, 'salinasA_corrected', 'salinasA_gt')

83


In [ ]:
def estAdditiveNoise(r, verbose = 'on'):
    verbose = 0
    small = 1e-6
    verbose = 0
    L, N = r.shape
    w = np.zeros((L, N))
    if verbose == 'on':
        print('computing the sample correlation matrix and its inverse')
    RR = np.dot(r, r.T) 
    if np.linalg.cond(RR + small * np.eye(L)) < 1e-8:
    # near singular, so we double perturbation size.
        small = 5e-5
    RRi = np.linalg.inv(RR + small * np.eye(L))
    if verbose:
        print("computing band")
    for i in range(1, L+1):
        if verbose:
            print('\b\b\b%3d',i)
        XX = RRi - (np.outer(RRi[:, i], RRi[i, :])) / RRi[i, i]
        RRa = RR[:, i].copy()
        RRa[i] = 0
        
        beta = np.dot(XX, RRa)
        beta[i] = 0
        # need to compute np.dot(beta', r) i.e., conjugate of beta - w(i,:) = r(i,:) - beta'*r;
        w[i, :] = r[i, :] - np.dot(np.conj(beta),r)
    if verbose == 'on':
        print('\ncomputing noise correlation matrix')
    Rw = np.diag(np.diag(np.dot(w, w.T) / N))
    return w, Rw

In [ ]:
def hysime(*args):
#     function [varargout]=hysime(varargin)
    """HySime: Hyperspectral signal subspace estimation

    [kf,Ek]=hysime(y,w,Rw,verbose);

    Input:
        y  hyperspectral data set (each column is a pixel)
            with (L x N), where L is the number of bands
            and N the number of pixels
        w  (L x N) matrix with the noise in each pixel
        Rw noise correlation matrix (L x L)
        verbose [optional] (on)/off
    Output
        kf signal subspace dimension
        Ek matrix which columns are the eigenvectors that span 
            the signal subspace

    Copyright: Jose Nascimento (zen@isel.pt)
                & 
                Jose Bioucas-Dias (bioucas@lx.it.pt)

    For any comments contact the authors"""


# # noise_type = 'additive'; % default value
# # verbose = 1; verb ='on'; % default value
# # for i=2:nargin 
# #    switch lower(varargin{i}) 
# #        case {'additive'}, noise_type = 'additive';
# #        case {'poisson'}, noise_type = 'poisson';
# #        case {'on'}, verbose = 1; verb = 'on';
# #        case {'off'}, verbose = 0; verb = 'off';
# #        otherwise, error('parameter [%d] is unknown',i);
# #    end
# # end
# # verbose =0;
    # error checking
    if len(args) < 1 or len(args) > 3:
        raise ValueError("Invalid number of input arguments. Expected between 1 and 3.")

    if len(args) > 2:
        raise ValueError("Too many output parameters")

    y = args[0]
    if not isinstance(y, np.ndarray):
        raise ValueError("The data set must be an L x N matrix")
    
    noise_type = 'additive'
    verbose = 1
    verb = 'on'
    
    for i in range(1, len(args)):
        arg = args[i].lower()
        if arg == 'additive':
            noise_type = 'additive'
        elif arg == 'poisson':
            noise_type = 'poisson'
        elif arg == 'on':
            verbose = 1
            verb = 'on'
        elif arg == 'off':
            verbose = 0
            verb = 'off'
        else:
            raise ValueError(f"Parameter [{i+1}] is unknown")
    verbose = 0

    
    L,N = y.shape
    if L<2:
        raise ValueError('Too few bands to estimate the noise.')

    if verbose:
        print('Noise estimates:')

    if (noise_type == 'poisson'):
        # sqy = np.sqrt(y*(y>0));     
        sqy = np.sqrt(np.multiply(y,(y>0)))    
        [u,Ru] = estAdditiveNoise(sqy,verb); % noise estimates
        x = (sqy - u).^2;            % signal estimates 
        w = sqrt(x).*u*2;
        Rw = w*w'/N; 
    else % additive noise
        [w,Rw] = estAdditiveNoise(y,verb); % noise estimates        
    end



# [Ln,Nn] = size(w);
# [d1,d2] = size(Rw);

# if Ln~=L || Nn~=N  % n is an empty matrix or with different size
#    error('empty noise matrix or its size does not agree with size of y\n'),
# end
# if (d1~=d2 || d1~=L)
#    fprintf('Bad noise correlation matrix\n'),
#    Rw = w*w'/N; 
# end    


# x = y - w;

# if verbose,fprintf(1,'Computing the correlation matrices\n');end
# [L,N]=size(y);
# Ry = y*y'/N;   % sample correlation matrix 
# Rx = x*x'/N;   % signal correlation matrix estimates 
# if verbose,fprintf(1,'Computing the eigen vectors of the signal correlation matrix\n');end
# [E,D]=svd(Rx); % eigen values of Rx in decreasing order, equation (15)
# dx = diag(D);

# if verbose,fprintf(1,'Estimating the number of endmembers\n');end
# Rw=Rw+sum(diag(Rx))/L/10^5*eye(L);

# Py = diag(E'*Ry*E); %equation (23)
# Rw = diag(E'*Rw*E); %equation (24)
# cost_F = -Py + 2 * Rw; %equation (22)
# kf = sum(cost_F<0);
# [dummy,ind_asc] = sort( cost_F ,'ascend');
# Ek = E(:,ind_asc(1:kf));
# if verbose,fprintf(1,'The signal subspace dimension is: k = %d\n',kf);end

# varargout(1) = {kf};
# if nargout == 2, varargout(2) = {Ek};end
# return


# function [w,Rw]=estAdditiveNoise(r,verbose) 

# verbose = 0;
# small = 1e-6;
# verbose = 0;
# [L, N] = size(r);
# % the noise estimation algorithm
# w=zeros(L,N);
# if verbose 
#    fprintf(1,'computing the sample correlation matrix and its inverse\n');
# end
# RR=r*r';     % equation (11)
# if rcond(RR+small*eye(L))<1e-8
#     % near singular, so we double perturbation size.
#     small = 5e-5;
# end
# RRi=inv(RR+small*eye(L)); % equation (11)
# if verbose, fprintf(1,'computing band    ');end
# for i=1:L
#     if verbose, fprintf(1,'\b\b\b%3d',i);end
#     % equation (14)
#     XX = RRi - (RRi(:,i)*RRi(i,:))/RRi(i,i);
#     RRa = RR(:,i); RRa(i)=0; % this remove the effects of XX(:,i)
#     % equation (9)
#     beta = XX * RRa; beta(i)=0; % this remove the effects of XX(i,:)
#     % equation (10)
#     w(i,:) = r(i,:) - beta'*r; % note that beta(i)=0 => beta(i)*r(i,:)=0
# end
# if verbose, fprintf(1,'\ncomputing noise correlation matrix\n');end
# Rw=diag(diag(w*w'/N));
# return

In [ ]:
def load_hyperparameters(HSI, HSIName, AlgName):
    # Define necessary variables and functions
    def hysime(X):
        # Define your hysime function here
        pass

    # Assume X, M, N, and AlgName are defined
    if AlgName == 'D-VIC':
        # Perform k-nearest neighbors search
        nbrs = NearestNeighbors(n_neighbors=1000, algorithm='auto').fit(X)
        Dist_NN, _ = nbrs.kneighbors(X)

        # Determine NN, pct, and K based on HSIName
        if HSIName == 'Synthetic Data':
            NN = 320
            pct = 10.5
            K = 3
        elif HSIName == 'Salinas A':
            NN = 30
            pct = 96.3157894736842
            K = 6
        elif HSIName == 'Jasper Ridge':
            NN = 20
            pct = 92.75862068965517
            K = 4
        elif HSIName == 'Indian Pines':
            NN = 40
            pct = 76.0526315789474
            K = 16

        # Set default parameters
        Hyperparameters = {}
        Hyperparameters['SpatialParams'] = {'ImageSize': (M, N)}
        Hyperparameters['NEigs'] = 10
        Hyperparameters['NumDtNeighbors'] = 200
        Hyperparameters['Beta'] = 2
        Hyperparameters['Tau'] = 10**(-5)
        Hyperparameters['Tolerance'] = 1e-8
        Hyperparameters['K_Known'] = K
        Hyperparameters['EndmemberParams'] = {
            'Algorithm': 'ManyAVMAX',
            'NumReplicates': 100,
            'K': hysime(X.T)  # Assuming hysime is defined elsewhere
        }
        Hyperparameters['DiffusionNN'] = NN
        Hyperparameters['DensityNN'] = NN  # Must be ≤ 1000
        Hyperparameters['Sigma0'] = np.percentile(Dist_NN[Dist_NN > 0], pct, interpolation='nearest')

        print(Hyperparameters)